### Entrenamiento de Modelos

En todos los pasos, imprimir los resultados sobre el conjunto de entrenamiento y sobre el conjunto de test. Usar siempre GridSearchCV (con kfold) para encontrar los mejores parámetros.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from sklearn.neural_network import MLPClassifier

In [2]:
data = pd.read_csv('../datasets/sneep-unificado-2002-2017-CURADO.csv')

/home/lucia/.anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,36,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Selección de atributos 

Comenzaremos usando el siguiente listado de atributos:

In [3]:
X = data[['delito1_id','provincia_id','genero_id','nacionalidad_id',
         'es_reincidente_id','anio_condenado','edad_al_ser_condenado',
          'nivel_instruccion_id','estado_civil_id']].astype(int)
y = data['duracion_condena_rango'].astype(int)

xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.10, random_state = 0)

#### Tareas a realizar:

1- Crear una función que permita almacenar en un Dataframe de Pandas los siguientes datos:
"id_ejecucion", "tipo_modelo", "descripcion", "accuracy", "precision", "recall". Para cada predicción sobre el modelo de test que realicemos cargar los resultados en este df, con el objetivo de tener un log de trabajo. Al finalizar guardar el df en un csv.

In [6]:
# Create the pandas DataFrame 
log = pd.DataFrame(columns = ["id_ejecución", "tipo_modelo", "descripcion", "accuracy", "precision", "recall"]) 
# Save it to .csv
log.to_csv("log.csv")

In [7]:
def save(num_ejec, y_pred, y_true=yTest, tipo_modelo=None, descripcion=None, df=log):
    """
    Función que guarda los datos del modelo
    """
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    
    df.loc[num_ejec] = ['ejec' + str(num_ejec)] + [tipo_modelo] + [descripcion] + [accuracy] + [precision] + [recall]
    
    df.to_csv("log.csv")
    
    print("Reporte para el modelo número " + str(num_ejec) + " :")
    print(classification_report(y_true, y_pred))
    print("Actualización de la tabla:")
    print(df)

También definimos una función para hacer la búsqueda de los mejores parámetros:

In [8]:
def grid(model, params):

    clf = GridSearchCV(model, params, cv=StratifiedKFold().split(xTrain, yTrain))
    clf.fit(xTrain, yTrain)

    print("Mejor conjunto de parámetros:")
    print(clf.best_params_, end="\n\n")

    print("Puntajes de la grilla:", end="\n\n")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("Exactitud: %0.4f (+/-%0.04f) para los parámetros %r" % (np.sqrt(mean), np.sqrt(std), params))
        
    return clf

2- Entrenar un modelo con LogisticRegression

In [9]:
i = 0 # En esta variable guardamos la cantidad de modelos ejecutados hasta el momento

In [ ]:
# Regresión Logística
model = LogisticRegression(random_state=0) # semilla para reproducibilidad
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (penalty=l2, C=1.0, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='LogisticRegression', descripcion=desc) 
i+=1

Buscamos mejores parámetros:

In [ ]:
logi_para = {'penalty':['l1','l2'], # l1 lasso l2 ridge
             'C':np.logspace(-3,3,7),
             'random_state':[0]
             }

model = grid(model = LogisticRegression(), params = logi_para)

Luego corremos el modelo con los mejores parámetros encontrados:

In [ ]:
best_params = model.best_params_

model = LogisticRegression(penalty=best_params['penalty'],
                           C=best_params['C'],
                           random_state=best_params['random_state']
                           )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "penalty={}, C={}, random_state={}"
desc = desc.format(str(best_params['penalty']),
                str(best_params['C']), 
                str(best_params['random_state'])
                )

save(num_ejec=i, y_pred=yPred, tipo_modelo='LogisticRegression', descripcion=desc)
i+=1

3- Entrenar un modelo de DecisionTreeClassifier

In [ ]:
# Árbol de decision
model = DecisionTreeClassifier(random_state=0)
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (criterion=gini, max_depth=None, max_features=None, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='DecisionTreeClassifier', descripcion=desc)
i+=1

Buscamos mejores parámetros:

In [ ]:
tree_para = {'criterion':['gini','entropy'],
             'max_depth':[None, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 100, 110, 120, 130, 150],
             'max_features':['auto', 'sqrt', 'log2', None],
             'random_state':[0]
            }

model = grid(model = DecisionTreeClassifier(), params = tree_para)

Corremos el modelo con los mejores parámetros encontrados:

In [ ]:
best_params = model.best_params_

model = DecisionTreeClassifier(criterion=best_params['criterion'], 
                               max_depth=best_params['max_depth'],
                               max_features=best_params['max_features'],
                               random_state=best_params['random_state']
                              )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "criterion={}, max_depth={}, max_features={}, random_state={}"
desc = desc.format(str(best_params['criterion']),
                   str(best_params['max_depth']),
                   str(best_params['max_features']), 
                   str(best_params['random_state'])
                  )

save(num_ejec=i, y_pred=yPred, tipo_modelo='DecisionTreeClassifier', descripcion=desc)
i+=1

4- Entrenar un modelo Random Forest

In [ ]:
# Random Forest
model = RandomForestClassifier(random_state=0) 
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (n_estimators=10, criterion=gini, max_depth=None, max_features=auto, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='RandomForestClassifier', descripcion=desc) 
i+=1

Buscamos mejores parámetros:

In [ ]:
rand_para = {'n_estimators':[5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 10, 80, 90, 100, 200, 300, 320, 350, 400, 450],
             'criterion':['gini', 'entropy'],
             'max_depth':[None, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 100, 110, 120, 130, 150, 200, 250],
             'max_features':['auto', 'sqrt', 'log2', None],
             'random_state': [0],
            }

model = grid(model = RandomForestClassifier(), params = rand_para)

(La celda anterior tardó aproximadamente un día en ejecutarse).

Corremos el modelo con los mejores parámetros encontrados:

In [ ]:
best_params = model.best_params_

model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                               criterion=best_params['criterion'],
                               max_depth=best_params['max_depth'],
                               max_features=best_params['max_features'],
                               random_state=best_params['random_state']
                              )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "n_estimators={}, criterion={}, max_depth={}, max_features={}, random_state={}"
desc = desc.format(str(best_params['n_estimators']),
                   str(best_params['criterion']),
                   str(best_params['max_depth']), 
                   str(best_params['max_features']),
                   str(best_params['random_state'])
                  )
                
save(num_ejec=i, y_pred=yPred, tipo_modelo='RandomForestClassifier', descripcion=desc)
i+=1

5- Entrenar un modelo con XGBoost
(Ejemplo: https://www.kaggle.com/rozester/xgboost-example-python)

In [ ]:
# XGBoost 
model = xgb.XGBClassifier(random_state=0)
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = 'default parameters (max_depth=3, learning_rate=0.1, n_estimators=100, random_state=0)'

save(num_ejec=i, y_pred=yPred, tipo_modelo='XGBClassifier', descripcion=desc) 
i+=1

Buscamos mejores parámetros:

In [ ]:
xgb_para = {'n_estimators':[5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 10, 80, 90, 100, 200, 300, 320, 340, 350, 400],
            'max_depth':[3, 6, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 100, 110, 120, 130, 150, 200, 300],
            'learning_rate':[0.01, 0.05, 0.1],
            'random_state':[0]
           }

model = grid(model = xgb.XGBClassifier(), params = xgb_para)

/home/lucia/.anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:629: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


(la anterior celda tarda días en ejecutarse, habría que enviarla al cluster)

Corremos el modelo con los mejores parámetros encontrados:

In [ ]:
best_params = model.best_params_

model = xgb.XGBClassifier(n_estimators=best_params['n_estimators'], 
                          max_depth=best_params['max_depth'],
                          learning_rate=best_params['learning_rate'],
                          random_state=best_params['random_state']
                         )
model.fit(xTrain, yTrain)

yPred = model.predict(xTest)

desc = "n_estimators={}, max_depth={}, learning_rate={}, random_state={}"
desc = desc.format(str(best_params['n_estimators']),
                str(best_params['max_depth']), 
                str(best_params['learning_rate']),
                str(best_params['random_state'])
                )

save(num_ejec=i, y_pred=yPred, tipo_modelo='XGBClassifier', descripcion=desc)
i+=1